In [42]:
import os

In [43]:
%pwd

'e:\\Chicken_Disease_classifier'

In [3]:
os.chdir('../')

In [44]:
%pwd

'e:\\Chicken_Disease_classifier'

In [45]:
import tensorflow as tf
from  pathlib import Path

In [46]:
model=tf.keras.models.load_model('artifacts/training/model.h5')

In [71]:
from dataclasses import dataclass

@dataclass(frozen=True)
class evaluation_config:
    path_of_model:Path
    training_data:Path
    all_params:dict
    param_image_size:list
    param_batch_size:int


In [72]:
from CHICKEN_DISEASE_CLASSIFIER.utils.common import read_yaml,save_json,create_directories
from CHICKEN_DISEASE_CLASSIFIER.constants import *

In [74]:
class Configuraion:

    def __init__(self,file_path=config_file_path,param_file=param_file_path):
        self.config=read_yaml(file_path)
        self.params=read_yaml(param_file)

        create_directories([self.config.artifacts_root])

    def get_validation_data_config(self)->evaluation_config:
        
        eval_config=evaluation_config(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion",
            all_params=self.params,
            param_image_size=self.params.IMAGE_SIZE,
            param_batch_size=self.params.BATCH_SIZE


        )
        return eval_config


In [1]:
class Evaluation:
    def __init__(self,config:evaluation_config):
        self.config=config

    def _valid_generator(self):

        data_generator_kwargs=dict(
            rescale=1./255,
            validation_split=0.30
        )

        dataflow_kwargs=dict(
            target_size=self.config.param_image_size[:-1],
            batch_size=self.config.param_batch_size,
            interpolation='bilinear',
            )
        valid_generaor=tf.keras.preprocessing.image.ImageDataGenerator(
            **data_generator_kwargs 
            )
        self.valid_generator=valid_generaor.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
            
        )
    @staticmethod
    def load_model(path:Path)->tf.keras.Model:
            return tf.keras.models.load_model(path)
        
    def evaluation_model(self):
            self.model=self.load_model(path=self.config.path_of_model)
            self._valid_generator()
            self.score=self.model.evaluate(self.valid_generator)

    def save_score(self):
        scores={'loss':self.score[0],"accuracy":self.score[1]}
        save_json(path=Path("scores.jason"),data=scores)
        

        



NameError: name 'evaluation_config' is not defined

In [78]:
try:
    config = Configuraion()
    val_config = config.get_validation_data_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation_model()
    evaluation.save_score()

except Exception as e:
   raise e

[2023-08-16 12:08:37,465: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-16 12:08:37,510: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-16 12:08:37,527: INFO: common: created directory at: artifacts]
Found 2418 images belonging to 4 classes.
152/152 [==============================] - 1767s 12s/step - loss: 2.1022 - accuracy: 0.8222
[2023-08-16 12:38:09,912: INFO: common: json file saved at: scores.jason]
